In [1]:
from vermouth.gmx.itp_read import read_itp
from vermouth.forcefield import ForceField
from vermouth.ffinput import read_ff
import re
import copy
from os.path import isfile
import MDAnalysis as mda
import numpy as np
from tqdm import tqdm
import zlib

# Go Bond Visualization for Coarse-Grained Simulations

This notebook demonstrates the workflow for generating visualizable Gō bonds in coarse-grained (CG) molecular simulations.  

It includes the following steps:

1. **Load force field and topology**  
   Reads Martini-based CG topologies and identifies virtual sites.

2. **Parse Go nonbonded parameters**  
   Determines intra- and inter-molecular Gō bonds from a custom `.itp` file.

3. **Map bonds onto molecular coordinates**  
   For each trajectory frame, calculates distances between atoms and determines if a bond should exist based on a cutoff (multiples of LJ minimum distance).

4. **Generate visualization files**  
   Converts bonds into TCL-compatible lists, compresses them, and writes `.z` files for dynamic visualization in VMD using `cg_bonds-v6.tcl`.

**Note:** This notebook is primarily for demonstration and visualization of go bonds.


In [2]:
def input_topol_reader(file):
    """
    Reads a GROMACS topology (.top) file and extracts header information.

    This function parses the topology file to identify:
    - Included ITP files (not core Martini ITPs)
    - Molecule definitions under [ molecules ]
    - System information under [ system ]
    
    It does **not** handle [ moleculetype ] directives inside the file; 
    if present, it raises an error and asks to split the system into separate ITPs.

    Parameters
    ----------
    file : str
        Path to the topology file (.top) to read.

    Returns
    -------
    dict
        Dictionary containing:
        - 'core_itps': list of included ITP files (strings)
        - 'system': list of lines under [ system ] section
        - 'molecules': list of dictionaries with keys 'name' and 'n_mols' for each molecule
    """
    inclusions = []  # Stores included .itp files
    molecules = []   # Stores molecule names and counts
    system = []      # Stores system block lines
    go = []          # (Unused, could be for Go bond info later)

    # Read the topology file line by line
    with open(file) as f:
        mols = False  # Flag to indicate we are inside [ molecules ] section
        sys = False   # Flag to indicate we are inside [ system ] section
        
        for line in f.readlines():
            # Capture included ITP files
            if '#include' in line:
                inclusions.append(line.split('"')[1])
            
            # Capture molecule lines in [ molecules ] section
            if mols:
                if (line.strip()[0] != ';') and (len(line.split()) > 0):
                    molecules.append({
                        'name': line.split()[0],    # Molecule name
                        'n_mols': line.split()[1]   # Number of molecules
                    })
            
            # Detect [ molecules ] section
            if 'molecules' in line:
                mols = True
                sys = False
                system.append(line)
            
            # Detect [ system ] section
            if 'system' in line:
                sys = True
            
            # Collect lines in [ system ] section
            if sys:
                system.append(line)
            
            # Check for [ moleculetype ] directives (unsupported)
            if 'moleculetype' in line:
                msg = ("Your topology file contains [ moleculetype ] directives, "
                       "which this file parser cannot process. "
                       "Please split your system into multiple itp files.")
                raise IOError(msg)

    # Combine extracted information into a dictionary
    topol_lines = {
        'core_itps': inclusions,
        'system': system,
        'molecules': molecules
    }

    return topol_lines


In [3]:
def output_str(pairs):
    """
    Convert a list of residue index pairs into a VMD-compatible string.

    Parameters
    ----------
    pairs : list of tuple
        List of tuples representing start and end residue indices.

    Returns
    -------
    str
        String in the format "{start1 end1} {start2 end2} ..." suitable for TCL scripts.
    """
    op_str = '{'
    for i in pairs:
        # Use unicode braces \u007b and \u007d for TCL compatibility
        op_str += f'\u007b{i[0]} {i[1]}\u007d '
    return op_str[:-1] + '}'  # Remove trailing space and close the string


def secondary_structure_parsing(lines, molname):
    """
    Parse secondary structure from ITP/molecule lines and generate VMD visualization commands.

    Parameters
    ----------
    lines : list of str
        Lines from a molecule .itp or force field file.
    molname : str
        Name of the molecule, used for output file naming.

    Behavior
    --------
    - Extracts header lines (starting with ';') for secondary structure info.
    - Identifies helices (H, G, I) and sheets (B, E) using regex.
    - Generates TCL commands for VMD to color helices and sheets, and exclude BB string.
    - Writes suggested commands to "<molname>_cgsecstruct.txt" if more than two helices or sheets exist.
    """
    header = []
    # Collect header lines starting with ';'
    for line in lines:
        if line[0] == ';':
            header.append(line[1:-1].strip())
        else:
            break

    ss_string = ''
    # Extract secondary structure string (all-uppercase)
    for line in header:
        if line.upper() == line:
            ss_string = line

    # Identify helices (3+ consecutive H/G/I) and sheets (3+ consecutive B/E)
    helices = [i.span() for i in re.finditer('([H|G|I]{3,})', ss_string)]
    sheets = [i.span() for i in re.finditer('([B|E]{3,})', ss_string)]

    # Build VMD exclude string for backbone
    vmd_excl_str = ''
    for i in helices + sheets:
        vmd_excl_str += f'(resid > {i[0]} and resid < {i[1]}) or '
    vmd_excl_str = vmd_excl_str[:-4]

    # Build color strings for helices and sheets
    hlx_col_str = ''
    for i in helices:
        hlx_col_str += f'(resid > {i[0]} and resid < {i[1]}) or '
    hlx_col_str = hlx_col_str[:-4]

    sht_col_str = ''
    for i in sheets:
        sht_col_str += f'(resid > {i[0]} and resid < {i[1]}) or '
    sht_col_str = sht_col_str[:-4]

    # Write TCL commands if sufficient helices/sheets exist
    if len(helices) > 2 or len(sheets) > 2:
        with open(f'{molname}_cgsecstruct.txt', 'w') as f:
            f.write("Suggested commands for viewing your molecule with cg_bonds-v6.tcl:\n")
            f.write(f'cg_helix {output_str(helices)} -hlxcolor "purple" -hlxfilled yes -hlxrad 3 '
                    f'-hlxmethod cylinder -hlxmat "AOChalky" -hlxres 50\n')
            f.write(f'cg_sheet {output_str(sheets)} -shtfilled "yes" -shtmat "AOChalky" -shtres 50 '
                    f'-shtcolor "red" -shtmethod flatarrow -shtarrwidth 5 -shtheadsize 10 -shtarrthick 3 -shtsides "sharp"\n')
            f.write('\nAdditionally, use the following command to remove the BB string from your molecule:\n')
            f.write(f'name BB and not ({vmd_excl_str})')
            f.write('\nThese commands will have to be modified to specify molid if you have multiple proteins in your system\n')
            f.write('\nAlternatively, use the following to just colour the backbone:')
            f.write(f'\nhelices: name BB and ({hlx_col_str})')
            f.write(f'\nsheets: name BB and ({sht_col_str})')


def _misc_file_reader(lines):
    """
    Handle miscellaneous ITP files that cannot be read by read_itp.

    Parameters
    ----------
    lines : list of str
        Lines from the input .itp file.

    Returns
    -------
    tuple or None
        - defines: dict mapping definition name to parameters (for bonded terms)
        - others: list of lines not containing '#define'
        Returns None if there is nothing to extract.
    """
    defines = {i.split()[1]: i.split(';')[0].split()[2:5] for i in lines if '#define' in i}
    others = [line for line in lines if '#define' not in line]
    if others == lines:
        return None
    else:
        return defines, others


def system_reading(topology):
    """
    Read a GROMACS topology file into a Vermouth ForceField object.

    Parameters
    ----------
    topology : str
        Path to the .top file.

    Returns
    -------
    ff : ForceField
        Vermouth ForceField object containing molecules and bonded interactions.
    topol_lines : dict
        Dictionary from input_topol_reader with core_itps, system, and molecules.
    system_defines : dict
        Dictionary of external #define bonded terms extracted from miscellaneous files.
    """
    print(f"Reading input {topology}")
    topol_lines = input_topol_reader(topology)

    # Read each core ITP file
    d = {}
    for i, itp_file in enumerate(topol_lines['core_itps']):
        with open(itp_file, encoding="utf-8") as f:
            d[i] = f.readlines()

    # Initialize ForceField
    ff = ForceField('martini3001')
    system_defines = {}

    for i, lines in d.items():
        try:
            # Try reading ITP into ForceField
            read_itp(lines, ff)
            secondary_structure_parsing(lines, list(ff.blocks)[-1])
        except OSError:
            # If the ITP cannot be read, handle miscellaneous definitions
            misc_result = _misc_file_reader(lines)
            if misc_result is None:
                # Ignore default files like martini_v3.0.0.itp
                if "[ defaults ]" not in [k.strip() for k in lines]:
                    print(f"Error reading {topol_lines['core_itps'][i]}. Will ignore and exclude from output system.")
            else:
                # Track #define terms and read remaining ITP content
                for key, value in misc_result[0].items():
                    system_defines[key] = value
                read_itp(misc_result[1], ff)

    return ff, topol_lines, system_defines


In [4]:
def molecule_editor(ff, topol_lines,
                    virtual_sites=True, ext=False,
                    elastic=False,
                    go=False, go_nb_file=''):
    """
    Edit molecules in a Vermouth ForceField to generate visualizable topologies.

    This function modifies the ForceField molecules to:
    - Keep only the interactions needed for visualization (bonds, constraints, pairs, virtual sites)
    - Convert constraints and pairs into bonds for visualization
    - Optionally handle virtual sites by creating bonds to their constructing atoms
    - Optionally incorporate Go-like interactions from a nonbonded Go file
    - Optionally write extended bond information to a text file

    Parameters
    ----------
    ff : ForceField
        Vermouth ForceField containing molecules and interactions.
    topol_lines : dict
        Dictionary of molecules read from the topology (output of input_topol_reader).
    virtual_sites : bool, default True
        Whether to include bonds for virtual sites.
    ext : bool, default False
        Whether to write extended bond lists to a text file.
    elastic : bool, default False
        Placeholder for elastic network handling (not implemented here).
    go : bool, default False
        Whether to incorporate Go-like nonbonded interactions.
    go_nb_file : str, default ''
        Path to Go nonbonded ITP file (required if go=True).

    Returns
    -------
    written_mols : list
        List of molecules that were written out (currently unused in the code).
    mol_bonds : dict
        Dictionary mapping molecule name to a list of Go bond pairs.
    mol_lens : dict
        Dictionary mapping molecule name to number of nodes (atoms) in the molecule.
    """
    # Interaction types to keep for visualization
    keep = ['bonds', 'constraints', 'pairs', 'virtual_sitesn',
            'virtual_sites2', 'virtual_sites3']
    print("Writing visualisable topology files")

    written_mols = []  # Track molecules written
    mol_bonds = {}     # Store Go bond pairs per molecule
    mol_lens = {}      # Store number of nodes per molecule

    for molname, block in ff.blocks.items():
        # Only process molecules present in the original topology
        try:
            assert molname in [i['name'] for i in topol_lines['molecules']]
        except AssertionError:
            continue

        bonds_list = []

        # Remove unwanted interactions from the block
        for interaction_type in list(block.interactions):
            if interaction_type not in keep:
                del block.interactions[interaction_type]

        # Remove meta information from bonds
        for bond in block.interactions['bonds']:
            bond.meta.clear()

        # Convert constraints without IFDEF to bonds
        for bond in list(block.interactions['constraints']):
            if bond.meta.get('ifndef'):
                # Remove constraints that are conditional
                block.remove_interaction('constraints', bond.atoms)
            else:
                # Convert constraints to high-force bonds
                block.add_interaction('bonds', bond.atoms, bond.parameters + ['10000'])
                block.remove_interaction('constraints', bond.atoms)

        # Convert pairs into bonds for visualization
        for bond in block.interactions['pairs']:
            block.add_interaction('bonds', bond.atoms, bond.parameters[:2] + ['10000'])
        del block.interactions['pairs']

        # Handle virtual sites
        go_dict = {}  # Map atom types to site indices for Go bonds
        for vs_type in ['virtual_sitesn', 'virtual_sites2', 'virtual_sites3']:
            if virtual_sites:
                for vs in block.interactions[vs_type]:
                    site = vs.atoms[0]
                    constructors = vs.atoms[1:]
                    block.nodes[site]['mass'] = 1  # ensure mass is non-zero
                    if go:
                        # Track virtual site for Go bonds
                        aname = block.nodes[site]['atomname']
                        atype = block.nodes[site]['atype']
                        if (aname.split('_')[0] == 'molecule') or (aname == 'CA'):
                            go_dict[atype] = site
                    else:
                        # Add bonds between site and constructing atoms
                        for constructor in constructors:
                            block.add_interaction('bonds', [site, constructor],
                                                  ['1', '1', '1000'], meta={"comment": "bonded virtual site"})
            if block.interactions[vs_type]:
                del block.interactions[vs_type]

        # Handle Go-like nonbonded interactions
        if go:
            if not isfile(go_nb_file):
                raise FileNotFoundError("Gō nonbonded itp does not exist. Specify using -gf")

            with open(go_nb_file, "r") as f:
                nb_lines = f.readlines()
            # Remove comments and blank lines
            nb_lines = [line.split(';')[0].split(' ') for line in nb_lines if '[' not in line]

            # Map Go interactions to actual site indices
            try:
                for i in nb_lines:
                    try:
                        assert i[0] in go_dict
                        assert i[1] in go_dict
                        bonds_list.append([go_dict[i[0]], go_dict[i[1]]])
                    except AssertionError:
                        pass
            except KeyError:
                pass

        # Convert block to molecule object
        mol_out = block.to_molecule()

        # Optionally write extended bond list
        if ext:
            ext_bonds_list = [i.atoms for i in block.interactions['bonds']]
            stout = ''.join([f'{i[0]}\t{i[1]}\n' for i in ext_bonds_list])
            with open(f'{molname}_bonds.txt', 'w') as bonds_list_out:
                bonds_list_out.write(stout)

        mol_bonds[molname] = bonds_list
        mol_lens[molname] = len(block.nodes)

    return written_mols, mol_bonds, mol_lens


In [5]:
"""
Load the molecule and determine potential Go bonds between virtual sites.

This code reads the topology and Go nonbonded parameter file, then generates two lists:
- inter_bonds: bonds between atoms in different molecules
- intra_bonds: bonds between atoms within the same molecule

These lists include the corresponding indices and multipliers for visualization or further analysis.
"""

# Create a ForceField object to hold the molecule and its interactions
ff = ForceField(name='martini_go')

# Read the topology and force field parameters
ff, topol_lines, system_defines = system_reading('topol.top')

# Copy the first molecule block for manipulation
mol = ff.blocks['molecule_0'].copy()

# Create mappings between node indices and atom types for virtual sites
idx_to_atype = {}  # node index -> atom type
atype_to_idx = {}  # atom type -> node index
start_of_vs = False

for node_idx, node_data in mol.nodes.items():
    # Only consider virtual site atoms (start with 'molecule_')
    if node_data['atype'].startswith('molecule_'):
        idx_to_atype[node_idx] = node_data['atype']
        atype_to_idx[node_data['atype']] = node_idx
        if not start_of_vs:
            start_of_vs = node_idx  # Track first virtual site index

# Path to Go nonbonded parameter file
go_nb_file = 'go_nbparams.itp'
bonds_list = []

# Read Go nonbonded parameter file
with open(go_nb_file, "r") as f:
    nb_lines = f.readlines()

# Split each line into parameters and metadata, ignoring sections
params = [line.split(';')[0].split(' ') for line in nb_lines if '[' not in line]
meta = [line.split(';')[1].split(' ') for line in nb_lines if '[' not in line]

# Initialize lists to store inter- and intra-molecular Go bonds
inter_bonds = []
intra_bonds = []

len_molecule_0 = len(mol.nodes)                # Number of nodes in one molecule
amount = int(topol_lines['molecules'][0]['n_mols'])  # Total number of molecules

# Loop through each Go bond definition
for meta_line, param_line in zip(meta, params):
    if meta_line[0] == 'inter':
        # Inter-molecular bonds: between molecules k and l
        atom1_type, atom2_type = param_line[0], param_line[1]
        multiplier = float(param_line[3]) * (2**(1/6))  # LJ minimum distance

        for k in range(amount):
            for l in range(amount):
                if k == l:
                    continue  # Skip intra-molecular case
                idx1 = atype_to_idx[atom1_type] + (k * len_molecule_0)
                idx2 = atype_to_idx[atom2_type] + (l * len_molecule_0)
                inter_bonds.append([idx1, idx2, multiplier])

    elif meta_line[0] == 'intra' and param_line[0][-1] == 'c':
        # Intra-molecular bonds: within the same molecule
        atom1_type, atom2_type = param_line[0], param_line[1]
        multiplier = float(param_line[3]) * (2**(1/6))

        for k in range(amount):
            idx1 = atype_to_idx[atom1_type] + k * len_molecule_0
            idx2 = atype_to_idx[atom2_type] + k * len_molecule_0
            intra_bonds.append([idx1, idx2, multiplier])


Reading input topol.top


In [6]:
"""
Check intra-molecular bonds frame-by-frame and generate a compressed .z file for visualization.

This code:
- Loads a molecular trajectory using MDAnalysis
- Checks for bonds between virtual sites based on a cutoff
- Converts bonds per atom into TCL list format
- Compresses the result for efficient storage
"""

# Load the trajectory: structure (.gro) and trajectory (.xtc)
u = mda.Universe("no_water.gro", "whole.xtc")
n_atoms = len(u.atoms)
n_frames = len(u.trajectory)

# List of intra-molecular bonds: each element is (i, j, k)
# i,j are atom indices, k is multiplier for the cutoff distance
all_bonds = intra_bonds  

# Distance cutoff (nm) for bond formation
cutoff = 1.2

# Store bonds for each atom over all frames
all_bonds_per_atom = []

# Loop over frames with progress bar
for ts in tqdm(u.trajectory, desc="Checking bond distances"):
    # Initialize empty list of neighbors for each atom in this frame
    frame_bonds = [[] for _ in range(n_atoms)]

    # Check each bond
    for i, j, k in all_bonds:
        pos_i = u.atoms[i].position  # Atom i coordinates in Å
        pos_j = u.atoms[j].position  # Atom j coordinates in Å
        distance = np.linalg.norm(pos_i - pos_j) / 10.0  # Convert Å to nm

        if distance <= float(k) * cutoff:
            # Bond exists: add symmetrically to both atoms
            frame_bonds[i].append(j)
            frame_bonds[j].append(i)

    # Add this frame's bond list to the global list
    all_bonds_per_atom.extend(frame_bonds)

# Convert each atom's neighbors to TCL list format
# Example for one atom: "{1 2 3}"
bond_lines = ["{" + " ".join(map(str, sorted(set(bond_list)))) + "}" for bond_list in all_bonds_per_atom]

# Flatten all atom lists across frames into a single string
tcl_list = " ".join(bond_lines)

# Compress and write to .z file for VMD/TCL visualization
compressed = zlib.compress(tcl_list.encode("utf-8"))
with open("intra_bonds.z", "wb") as f:
    f.write(compressed)


Checking bond distances:   4%|█████▌                                                                                                                                      | 40/1001 [00:05<02:12,  7.28it/s]


KeyboardInterrupt: 

In [7]:
"""
Check inter-molecular bonds frame-by-frame and generate a compressed .z file for visualization.

This code:
- Loads a molecular trajectory using MDAnalysis
- Checks for bonds between virtual sites in different molecules based on a cutoff
- Ensures no atom exceeds a maximum number of bonds (here 12)
- Converts bonds per atom into TCL list format
- Compresses the result for efficient storage
"""

# Load the trajectory: structure (.gro) and trajectory (.xtc)
u = mda.Universe("no_water.gro", "whole.xtc")
n_atoms = len(u.atoms)
n_frames = len(u.trajectory)

# List of inter-molecular bonds: each element is (i, j, k)
# i,j are atom indices, k is multiplier for the cutoff distance
all_bonds = inter_bonds  

# Distance cutoff (nm) for bond formation
cutoff = 2.0

# Store bonds for each atom over all frames
all_bonds_per_atom = []

# Counters for debugging potential problem bonds
good = 0
bad = 0

# Loop over frames with progress bar
for ts in tqdm(u.trajectory, desc="Checking bond distances"):
    # Initialize empty set of neighbors for each atom in this frame
    frame_bonds = [set() for _ in range(n_atoms)]

    # Check each inter-molecular bond
    for i, j, k in all_bonds:
        pos_i = u.atoms[i].position  # Atom i coordinates in Å
        pos_j = u.atoms[j].position  # Atom j coordinates in Å
        distance = np.linalg.norm(pos_i - pos_j) / 10.0  # Convert Å to nm

        if distance <= float(k) * cutoff:
            # Add bond if both atoms have fewer than 12 existing bonds
            if len(frame_bonds[i]) < 12 and len(frame_bonds[j]) < 12:
                frame_bonds[i].add(j)
                frame_bonds[j].add(i)
            else:
                # Attempt to redirect to adjacent atoms if possible
                if (i - 1 < n_atoms and j - 1 < n_atoms and
                    len(frame_bonds[i - 1]) < 12 and len(frame_bonds[j - 1]) < 12):
                    frame_bonds[i - 1].add(j - 1)
                    frame_bonds[j - 1].add(i - 1)
                else:
                    # Give up adding this bond if redirect also fails
                    print('Too many bonds on atom', i, 'and/or', j)
                    continue

    # Add this frame's bond list to the global list
    all_bonds_per_atom.extend(frame_bonds)

# Convert each atom's neighbors to TCL list format
# Example for one atom: "{1 2 3}"
bond_lines = ["{" + " ".join(map(str, sorted(set(bond_list)))) + "}" for bond_list in all_bonds_per_atom]

# Flatten all atom lists across frames into a single string
tcl_list = " ".join(bond_lines)

# Compress and write to .z file for VMD/TCL visualization
compressed = zlib.compress(tcl_list.encode("utf-8"))
with open("inter_bonds.z", "wb") as f:
    f.write(compressed)


Checking bond distances: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1001/1001 [02:23<00:00,  6.98it/s]
